In [2]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from joblib import dump, load
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split
import pandas as pd


def preprocess_data(data, target_column, save_path, file_path):
    # Menentukan fitur numerik dan kategoris
    numeric_features = data.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_features = data.select_dtypes(include=['object']).columns.tolist()
    column_names = data.columns
    # Mengambil nama kolom tanpa target
    column_names = data.columns.drop(target_column)
    
    # Membuat DataFrame kosong dengan nama kolom
    df_header = pd.DataFrame(columns=column_names)
    
    # Menyimpan nama kolom sebgai header tanpa data
    df_header.to_csv(file_path, index=False)
    print(f"Nama kolom berhasil disimpan ke: {file_path}")
    
    # Pastikan target_column tidak ada di numeric_features atau categorical features
    if target_column in numeric_features:
        numeric_features.remove(target_column)
    if target_column in categorical_features:
        categorical_features.remove(target_column)
        
    # Pipeline untuk fitur numerik
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Pipeline untuk fitur kategoris
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('encoder', OrdinalEncoder())
    ])
    
    # Column Transformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )
    
    # Memisahkan target
    X = data.drop(columns=[target_column])
    y = data[target_column]
    
    # Membagi data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Fitting dan transformasi data pada training set
    X_train = preprocessor.fit_transform(X_train)
    # Transformasi pada data testing
    X_test = preprocessor.transform(X_test)
    
    # Simpan pipeline
    dump(preprocessor, save_path)
    
    return X_train, X_test, y_train, y_test 

In [2]:
data = pd.read_csv("personality_dataset.csv")
data.head()

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,4.0,No,4.0,6.0,No,13.0,5.0,Extrovert
1,9.0,Yes,0.0,0.0,Yes,0.0,3.0,Introvert
2,9.0,Yes,1.0,2.0,Yes,5.0,2.0,Introvert
3,0.0,No,6.0,7.0,No,14.0,8.0,Extrovert
4,3.0,No,9.0,4.0,No,8.0,5.0,Extrovert


In [12]:
X_train, X_test, y_train, y_test = preprocess_data(data, 'Personality', 'preprocessor_pipeline.joblib', 'data.csv')

Nama kolom berhasil disimpan ke: data.csv


In [4]:
# Validasi memastikan tahapan preprocessing dilakukan dan disimpan dengan baik
def inference(new_data, load_path):
    # Memuat pipeline preprocessing
    preprocessor = load(load_path)
    print(f"Pipeline preprocessing dimuat dari: {load_path}")
    
    # Transformasi data baru
    transformed_data = preprocessor.transform(new_data)
    return transformed_data

In [45]:
def inverse_transform_data(transformed_data, load_path, new_data_columns, num_cols, cat_cols):
    preprocessor = load(load_path)
    numeric_transformer = preprocessor.named_transformers_['num']['scaler']
    categorical_transform = preprocessor.named_transformers_['cat']['encoder']
    n_num = len(num_cols)
    n_cat = len(cat_cols)

    # Slicing sesuai urutan pipeline (asumsikan num_cols dulu, lalu cat_cols)
    transformed_numeric_data = transformed_data[:, :n_num]
    transformed_categorical_data = transformed_data[:, n_num:n_num+n_cat]

    original_numeric_data = numeric_transformer.inverse_transform(transformed_numeric_data)
    original_categorical_data = categorical_transform.inverse_transform(transformed_categorical_data)

    # Buat dataframe kosong dengan urutan kolom asli
    inversed_data = pd.DataFrame(index=range(transformed_data.shape[0]), columns=new_data_columns)

    # Isi kolom satu per satu sesuai urutan aslinya
    num_idx = 0
    cat_idx = 0
    for col in new_data_columns:
        if col in num_cols:
            inversed_data[col] = original_numeric_data[:, num_idx]
            num_idx += 1
        elif col in cat_cols:
            inversed_data[col] = original_categorical_data[:, cat_idx]
            cat_idx += 1

    return inversed_data

In [48]:
import numpy as np
pipeline_path = 'preprocessor_pipeline.joblib'
col = pd.read_csv("data.csv")

new_data = [4.0, "No", 6.0, 2.0, "No", 0.0, 8.0]

# Mengubah menjadi numpy.ndarray
new_data = np.array(new_data)

new_data = pd.DataFrame([new_data], columns=col.columns)
transformed_data = inference(new_data, pipeline_path)

num_cols = ['Time_spent_Alone',	'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']
cat_cols = ['Stage_fear', 'Drained_after_socializing']

inversed_data = inverse_transform_data(transformed_data, pipeline_path, new_data.columns, num_cols, cat_cols)

# Output hasil preprocessing dan inference
print("Data setelah preprocessing (training):")
print(new_data)
print("\nData baru setelah transformasi:")
print(transformed_data)
print("\nData setelah inverse transform:")
print(inversed_data)

Pipeline preprocessing dimuat dari: preprocessor_pipeline.joblib
Data setelah preprocessing (training):
  Time_spent_Alone Stage_fear Social_event_attendance Going_outside  \
0              4.0         No                     6.0           2.0   

  Drained_after_socializing Friends_circle_size Post_frequency  
0                        No                 0.0            8.0  

Data baru setelah transformasi:
[[-0.15866558  0.7312455  -0.43888178 -1.46781052  1.55593223  0.
   0.        ]]

Data setelah inverse transform:
   Time_spent_Alone Stage_fear  Social_event_attendance  Going_outside  \
0               4.0         No                      6.0            2.0   

  Drained_after_socializing  Friends_circle_size  Post_frequency  
0                        No                  0.0             8.0  
